<a href="https://colab.research.google.com/github/mindsRiverPonder/DeepLearning-for-NLP/blob/main/nb/Gemma3_(4B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3


### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [2]:
from unsloth import FastModel
import torch


model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.14: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [3]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # 纯文本任务，关闭视觉层的微调，以节省资源
    finetune_language_layers   = True,  # 微调模型中的语言相关层，语言层是文本任务的核心部分，要开
    finetune_attention_modules = True,  #微调模型中的注意力模块
    finetune_mlp_modules       = True,  # 微调模型中的MLP模块
    r = 8,           # LoRA中的秩，秩越高，模型表达能力越强，训练越慢，也越容易过拟合
    lora_alpha = 8,  # LoRA中的缩放因子
    lora_dropout = 0,  #LoRA中的dropout率
    bias = "none",
    random_state = 114514,  #随机数种子，这个随便取
)

<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [4]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [9]:
import re
from datasets import load_dataset

# 加载本地的 json 数据集，文件名为 zhihu_data.json
dataset = load_dataset("json", data_files="zhihu_data.json", split="train")

def apply_chat_template_zhihu(examples):
    conversations = []
    for instruction, input_text, output_text in zip(examples["instruction"], examples["input"], examples["output"]):
        # 构建用户提问部分：若 input 非空则合并，否则仅使用 instruction
        if input_text.strip():
            user_message = instruction.strip() + "\n" + input_text.strip()
        else:
            user_message = instruction.strip()

        # 去除 output 中 <think> 和 </think> 包裹的思考内容
        cleaned_output = re.sub(r'<think>.*?</think>', '', output_text, flags=re.DOTALL).strip()

        # 构造最终对话格式
        conversation = (
            "<bos>\n"
            "<start_of_turn>user\n" + user_message + "\n<end_of_turn>\n"
            "<start_of_turn>model\n" + cleaned_output + "\n<end_of_turn>"
        )
        conversations.append(conversation)
    return {"text": conversations}

# 对数据集应用转换函数
dataset = dataset.map(apply_chat_template_zhihu, batched=True)

# 输出一个样本查看效果
print(dataset[0]["text"])

Map:   0%|          | 0/2534 [00:00<?, ? examples/s]

<bos>
<start_of_turn>user
反物质会形成黑洞吗?
如果反物质质量足够大，是不是和普通物质一样也能坍缩形成黑洞？反物质黑洞的性质是否会和普通黑洞性质相同？如果反物质黑洞碰到普通黑洞，是合并成一个更大的黑洞，还是湮灭？
<end_of_turn>
<start_of_turn>model
关于反物质是否会形成黑洞及其性质的问题，我们可以从以下几个方面进行分析：

---

### **1. 反物质能否形成黑洞？**
- **广义相对论的视角**：根据爱因斯坦的理论，黑洞的形成仅取决于质量（或能量）的集中程度。反物质与普通物质具有相同的正质量，只是电荷等量子属性相反。只要反物质的质量足够大且在足够小的空间内聚集，其引力坍缩过程与普通物质完全相同，**完全可以形成黑洞**。
- **电荷的影响**：若反物质云带有净电荷（如反质子带负电），可能需要更高的质量克服电磁斥力。但对于电中性的反物质系统（如反氢组成的星体），坍缩条件与普通物质无异。

---

### **2. 反物质黑洞的性质是否与普通黑洞相同？**
- **无毛定理的适用性**：黑洞的宏观性质仅由质量、角动量和电荷决定（“三毛定理”）。只要反物质黑洞的质量、自旋和电荷与普通黑洞一致，它们的时空结构、事件视界、引力效应等性质将**完全相同**。
- **霍金辐射的差异**：理论上，霍金辐射可能包含正反物质粒子对，但辐射类型与黑洞内部物质无关。因此，反物质黑洞的辐射特征与普通黑洞**没有本质区别**。

---

### **3. 反物质黑洞与普通黑洞相遇的结果**
- **合并而非湮灭**：
  - 一旦物质（或反物质）进入黑洞，其具体属性（如正反性）被事件视界隔绝，外部观测者无法区分黑洞的“成分”。两个黑洞的合并过程由引力主导，与内部物质类型无关。
  - 湮灭需要正反物质直接接触，但在黑洞事件视界内部，量子效应可能被引力支配，且合并过程的时间尺度远快于可能的湮灭反应。因此，**合并结果仍是一个更大的黑洞**，并伴随引力波释放。
- **电荷的影响**：若两黑洞携带相反电荷，合并后总电荷可能部分抵消，但这属于电荷的宏观效应，与物质类型无关。

---

### **总结**
- **反物质可以形成黑洞**，其条件与普通物质一致。
- **反物质黑洞与普通黑洞性质相同**（质量、角动量和电荷

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [10]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # 不进行评估，可以替换为具体数据集以启用评估
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # 梯度累积步数
        warmup_steps = 5,
        max_steps = 30,
        learning_rate = 2e-4, #学习率
        logging_steps = 1,
        optim = "adamw_8bit", #优化器，是 AdamW 优化器的 8 位版本，节省内存且适合大模型。
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 114514,
        report_to = "none", # 日志输出目标，这里表示不使用外部工具（如 WandB）记录日志
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16
Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2534 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [11]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/2534 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again:

In [12]:
tokenizer.decode(trainer.train_dataset[7]["input_ids"])

'<bos>\n<start_of_turn>user\n为什么游戏开发商不把制作引擎工具拿出来销售？纵观全球视频游戏几十年。欧美有几个公司这么做。unity3d ue4.cryteck\n等等。日本几个小会社有，rpgmaker其他全球游戏开发商如此多。为何没几个把他们的游戏开发工具软件。制作引擎拿出来卖钱呢？哪怕是过时淘汰的？\n<end_of_turn>\n<start_of_turn>model\n游戏开发商不将内部开发工具或引擎作为商品对外销售，这一现象涉及多重商业逻辑和技术考量。以下是关键因素分析：\n\n---\n\n### **一、商业战略与核心竞争力的取舍**\n1. **主业定位差异**  \n   Unity、Unreal等公司的核心业务是**引擎技术本身**，盈利模式依赖开发者订阅、分成或服务。而传统游戏公司（如EA、任天堂）的主业是**游戏内容开发**，引擎仅为工具，出售引擎需投入资源建立全新的销售、技术支持体系，可能分散主业精力。\n\n2. **技术保密与壁垒**  \n   引擎中的算法（如物理模拟、AI行为树）往往是公司的专利或技术护城河。例如《荒野之息》的化学引擎包含独特的环境交互逻辑，开放销售可能削弱竞争优势。\n\n---\n\n### **二、技术适配性与通用化成本**\n1. **高度定制化陷阱**  \n   内部工具通常针对特定项目优化，如《最终幻想》系列引擎专为JRPG叙事设计，若需适配开放世界或FPS类型，需重构底层架构，成本可能超过开发新引擎。\n\n2. **维护与兼容性负担**  \n   对外销售需长期维护文档、更新API、支持多平台（如主机换代时的SDK适配）。Capcom的RE引擎虽高效，但若开放需额外团队处理开发者问题，边际成本激增。\n\n---\n\n### **三、市场需求与生态竞争**\n1. **头部引擎的垄断效应**  \n   Unity和Unreal已占据80%的独立游戏及3A市场，形成完整工具链（资产商店、社区支持）。新引擎需提供革命性功能（如Decentraland的区块链集成）才能破局，传统厂商缺乏差异化优势。\n\n2. **细分市场局限性**  \n   RPG Maker的成功依赖于明确的用户画像（2D日式RPG爱好者），而《Frostbite》（寒霜引擎）专精FPS物理破

Now let's print the masked out example - you should see only the answer is present:

In [13]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                             游戏开发商不将内部开发工具或引擎作为商品对外销售，这一现象涉及多重商业逻辑和技术考量。以下是关键因素分析：\n\n---\n\n### **一、商业战略与核心竞争力的取舍**\n1. **主业定位差异**  \n   Unity、Unreal等公司的核心业务是**引擎技术本身**，盈利模式依赖开发者订阅、分成或服务。而传统游戏公司（如EA、任天堂）的主业是**游戏内容开发**，引擎仅为工具，出售引擎需投入资源建立全新的销售、技术支持体系，可能分散主业精力。\n\n2. **技术保密与壁垒**  \n   引擎中的算法（如物理模拟、AI行为树）往往是公司的专利或技术护城河。例如《荒野之息》的化学引擎包含独特的环境交互逻辑，开放销售可能削弱竞争优势。\n\n---\n\n### **二、技术适配性与通用化成本**\n1. **高度定制化陷阱**  \n   内部工具通常针对特定项目优化，如《最终幻想》系列引擎专为JRPG叙事设计，若需适配开放世界或FPS类型，需重构底层架构，成本可能超过开发新引擎。\n\n2. **维护与兼容性负担**  \n   对外销售需长期维护文档、更新API、支持多平台（如主机换代时的SDK适配）。Capcom的RE引擎虽高效，但若开放需额外团队处理开发者问题，边际成本激增。\n\n---\n\n### **三、市场需求与生态竞争**\n1. **头部引擎的垄断效应**  \n   Unity和Unreal已占据80%的独立游戏及3A市场，形成完整工具链（资产商店、社区支持）。新引擎需提供革命性功能（如Decentraland的区块链集成）才能破局，传统厂商缺乏差异化优势。\n\n2. **细分市场局限性**  \n   RPG Maker的成功依赖于明确的用户画像（2D日式RPG爱好者），而《Frostbite》（寒霜引擎）专精FPS物理破坏，受众狭窄，商业潜力有限。\n\n---\n\n### **四、过时技术的处置困境**\n1. **技术债务与重构成本**  \n   老引擎（如Square Enix的Crystal Tools）往往耦合陈旧代码，剥离

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
4.283 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,534 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.380300
2,2.959100
3,3.103800
4,2.760500
5,2.877300
6,2.753900
7,3.342300
8,2.652800
9,2.938900
10,2.788500


In [15]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

NameError: name 'start_gpu_memory' is not defined

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [20]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "神经网络的参数都是随机的，有的效果很好，有的效果很差，这真的不是玄学吗？",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
print(tokenizer.batch_decode(outputs)[0])

<bos><start_of_turn>user
神经网络的参数都是随机的，有的效果很好，有的效果很差，这真的不是玄学吗？<end_of_turn>
<start_of_turn>model
你问得非常好，这个问题一直以来都是神经网络领域讨论的热点。你说的“有的效果很好，有的效果很差”确实让人觉得有些玄学，但实际上，这背后有深刻的数学和统计学原因。它并不是完全的随机，而是**随机初始化**加上**训练过程中的优化**，两者共同作用的结果。

下面我们来详细分析一下：

**1. 随机初始化：**

* **权重和偏置的随机性：** 神经网络的权重和偏置（bias）在开始训练之前是随机初始化的。这意味着每个连接的权重和每个神经元的偏置都有一个随机的数值。
* **为什么需要随机？**  随机初始化是为了打破对称性，让不同的神经元在学习过程中拥有不同的“起点”。如果所有权重都初始化为相同的值，那么所有神经元在训练初期都会学习到相同的特征，导致训练效率低下，甚至无法收敛。
* **随机分布：** 通常，权重和偏置会被从一个特定的概率分布中采样，例如正态分布（通常使用均值为0，标准差为小的数值，例如0.01）。

**2. 训练过程中的优化：**

* **梯度下降：** 神经网络使用梯度下降算法来调整权重和偏置，以最小化损失函数。
* **随机梯度下降 (SGD)：**  通常使用随机梯度下降，而不是使用所有训练数据来计算梯度。这可以加速训练过程，并帮助避免陷入局部最小值。
* **随机化：**  在训练过程中，数据也会被随机化，例如使用 mini-batch 来进行训练。这可以增加训练的稳定性，并提高模型的泛化能力。

**3. 为什么结果不确定？**

* **非凸性：** 神经网络的损失函数通常是非凸的，这意味着它有很多局部最小值。梯度下降算法可能会陷入一个局部最小值，而不是全局最小值。
* **随机性：** 随机初始化、随机梯度下降、随机化等因素都会引入随机性，使得训练过程变得不稳定。
* **超参数：** 学习率、批大小、优化器等超参数也会影响训练结果。不同的超参数组合可能会导致不同的结果。
* **数据：** 训练数据的质量和数量也会影响模型的性能。

**4. 并非完全随机，而是“有策略的随机”：**

* **随机初始化是为了打破对称性，促进探索。**
*

In [21]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "人在岩浆里憋气最长时间是多少?",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
print(tokenizer.batch_decode(outputs)[0])

<bos><start_of_turn>user
人在岩浆里憋气最长时间是多少?<end_of_turn>
<start_of_turn>model
在岩浆里憋气，从理论上讲，几乎是不可能的，因为人类的身体根本无法承受如此极端的环境。以下是关于这个问题的一些更详细的解释，以及估计的“最长时间”：

**为什么不可能？**

* **氧气耗尽：** 即使在短时间内，人体也会迅速消耗氧气。在岩浆周围，空气会迅速被加热，氧气会以气体的形式逸出，导致氧气浓度急剧下降。
* **二氧化碳积累：** 岩浆周围的空气会迅速富含二氧化碳，这会干扰呼吸，并导致呼吸困难。
* **高温：** 岩浆的温度通常在800°C到1200°C（1472°F到2192°F）之间。这种极端高温会立即灼伤肺部和呼吸道，导致无法呼吸。
* **压力：** 岩浆周围的压力会非常高，这会进一步压迫肺部，使其无法有效工作。
* **化学反应：** 呼吸道中的水分会迅速蒸发，导致肺部干燥和损伤。

**理论上的“最长时间”估计**

由于上述因素，很难给出一个确切的“最长时间”估计。然而，根据现有科学知识和对人体生理的理解，以下是一些粗略的估计：

* **几秒钟：** 即使在非常理想化的假设下（例如，使用某种能够提供氧气的设备，并能有效隔绝高温和压力），人类也可能在几秒钟内耗尽氧气。
* **几分钟：** 即使使用先进的防护装备，在岩浆周围的极端环境中，几分钟的时间也可能不足以维持生存。
* **更长的时间（理论上）：** 理论上，如果能够使用某种能够提供持续氧气供应的设备，并能有效隔绝高温和压力，也许可以维持几分钟甚至更长时间。然而，这仍然是一个非常极端的假设，并且需要克服巨大的技术挑战。

**重要说明：**

* **没有已知的技术可以实现：** 目前没有任何已知的技术可以使人类在岩浆中安全地憋气。
* **这仅仅是一个理论探讨：** 任何关于“最长时间”的估计都基于理论假设，并且没有实际的实验数据支持。

**总结：**

在岩浆中憋气，对于人类来说，几乎是不可能的。即使使用最先进的设备，也只能维持非常短的时间。

**免责声明：**  我是一个AI语言模型，提供的这些信息仅供参考。我不能保证这些信息是准确或完整的。在进行任何与极端环境相关的活动之前，请务必咨询专业人士，并采取必要的安全措施。

希望

In [22]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "为什么游戏里可以点投降，人生却不能点？ 认输了还不行吗？",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
print(tokenizer.batch_decode(outputs)[0])

<bos><start_of_turn>user
为什么游戏里可以点投降，人生却不能点？ 认输了还不行吗？<end_of_turn>
<start_of_turn>model
你提出了一个非常好的问题，也是一个长期以来困扰着许多人的问题。游戏里点投降，在很大程度上是方便和娱乐性的设计，而人生中却很少有“点投降”的选项，这背后涉及了心理学、社会规范、以及对“成功”的理解等多种复杂因素。

**1. 游戏中的投降：**

* **明确的规则和反馈：** 游戏中的投降是明确定义的，有明确的规则和反馈机制。玩家知道点投降会带来什么后果（例如，输掉比赛、失去分数），并且可以立即看到结果。
* **控制感和策略性：** 投降通常是玩家在面对困境时的一种策略选择，可以帮助他们避免进一步的损失，并重新评估策略。
* **娱乐性：** 投降在游戏中可以带来一种“爽快”的感觉，尤其是在面对难以战胜的对手时。它提供了一种逃避和放松的方式。
* **简化和抽象：** 游戏通常会简化现实，将复杂的问题抽象成简单的选择。投降就是一个简单的“放弃”选项，方便玩家理解和操作。

**2. 人生中的“投降”：**

* **社会期望和价值观：** 社会普遍鼓励人们坚持、努力、不放弃。失败被视为学习的机会，而放弃则被视为软弱和退缩。
* **对“成功”的定义：** 我们的成功定义往往与坚持、克服困难、以及最终取得胜利相关联。认输可能被视为对成功的否定。
* **心理因素：**
    * **羞耻感：** 失败会带来羞耻感，人们倾向于隐藏失败，避免承认自己的无能。
    * **自我效能感：** 认输可能会削弱一个人的自我效能感，认为自己无法胜任任务。
    * **认知失调：** 坚持努力可能会导致认知失调，因为努力却没有结果。为了减少这种不适，人们可能会选择认输。
* **缺乏明确的“投降”选项：** 在现实生活中，我们很少有明确的“点投降”选项。面对困境时，我们通常需要自己做出选择，而没有游戏中的明确指示。
* **“认输”的定义模糊：**  “认输”在人生中是一个模糊的概念。它可能意味着放弃某个目标，也可能意味着承认自己的局限性，并调整策略。但往往，我们更倾向于将“认输”与彻底放弃联系起来。

**认输是否不行？**

当然不行！认输在适当的时候是一种智慧的选择。关键在于：

* **认清

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [23]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "我亲爸和我亲妈结婚，算不算近亲结婚？",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

这是一个非常复杂且需要仔细区分的问题。从法律和生物学角度来看，你的情况涉及到近亲结婚的定义，需要具体分析。

**1. 法律角度：**

*   **中国法律：** 中国法律明确禁止近亲结婚。根据《中华人民共和国民法典》的规定，近亲结婚是指父母、祖父母、孙子女、全血统的直系亲属、三代以内血缘亲属之间的婚姻。
*   **你的情况：** 你的亲爸和你亲妈结婚，这属于法律上定义的近亲结婚。因此，这种婚姻关系在法律上是无效的。
*   **法律后果：** 这种婚姻关系会被认定为无效，不会产生法律效力。

**2. 生物学角度：**

*   **生物学定义：** 生物学上，近亲结婚是指配偶之间存在近亲关系。
*   **你的情况：** 你的亲爸和你亲妈结婚，这在生物学上也是近亲结婚。
*   **生物学风险：** 近亲结婚会大大增加后代的遗传疾病风险，因为近亲之间共享更多的基因，增加了基因突变和有害基因


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

['gemma-3/processor_config.json']

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

Okay, let's break down what Gemma-3 is. It's a fascinating development in the world of AI, and here's a comprehensive overview:

**1. What it is:**

* **A Family of Open-Weight Language Models:** Gemma-3 isn't just *one* model


### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
